In [ ]:
import os
import glob
import pickle
from tqdm import tqdm
import numpy as np

# --- MODIFICA 1: Importa EfficientNetV2B0 e il suo preprocess_input ---
from tensorflow.keras.applications import EfficientNetV2B0
# Devi importare la funzione dal modulo specifico del modello che stai usando
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

SyntaxError: invalid syntax (2578589384.py, line 9)

In [13]:
# --- MODIFICA 2: Carica il modello EfficientNetV2B0 ---
print("Caricamento del modello EfficientNetV2B0...")
# include_top=False: rimuove il layer di classificazione finale
# pooling='avg': aggiunge un layer di Global Average Pooling per ottenere un vettore di feature
base_model = EfficientNetV2B0(
    weights='imagenet', 
    include_top=False, 
    pooling='avg'
)

# Il modello per l'estrazione è il modello base stesso
model = base_model
# La dimensione dell'input per EfficientNetV2B0 è 224x224
input_size = (224, 224) 
# La dimensione del vettore di output di EfficientNetV2B0 è 1280
output_feature_size = 1280

print(f"Modello per l'estrazione delle feature creato. Dimensione input: {input_size}, Dimensione output: {output_feature_size}")

Caricamento del modello EfficientNetV2B0...
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Modello per l'estrazione delle feature creato. Dimensione input: (224, 224), Dimensione output: 1280


In [ ]:
# --- MODIFICA 3: Aggiorna la funzione di estrazione ---
def extract_feature(img_path, model, target_size):
    """
    Carica un'immagine, la preprocessa per EfficientNetV2 e ne estrae le feature.
    """
    try:
        # Carica l'immagine con la dimensione richiesta dal modello
        img = image.load_img(img_path, target_size=target_size)
        
        # Converte l'immagine in un array NumPy
        x = image.img_to_array(img)
        
        # Aggiunge una dimensione per il "batch"
        x = np.expand_dims(x, axis=0)
        
        # USA IL PREPROCESS_INPUT SPECIFICO DI EFFICIENTNET
        x = preprocess_input(x)
        
        # Esegue l'estrazione delle feature
        feature = model.predict(x, verbose=0)
        
        # L'output di GlobalAveragePooling è già un vettore (1, 1280),
        # quindi .flatten() funziona perfettamente.
        return feature.flatten() 

    except Exception as e:
        print(f"\nErrore durante l'elaborazione di {img_path}: {e}")
        return None


In [14]:
# Specifica il percorso delle immagini
images_path = r"..\data\Flickr8k_Dataset\Images" 
if not os.path.exists(images_path):
    raise FileNotFoundError(f"La cartella delle immagini non è stata trovata in: {images_path}")

image_files = glob.glob(os.path.join(images_path, "*.jpg"))
print(f"Trovate {len(image_files)} immagini da processare.")

Trovate 8091 immagini da processare.


In [ ]:
# --- MODIFICA 4: Salva le nuove feature in un file separato ---
# È una buona pratica non sovrascrivere le vecchie feature!
output_folder = "features"
os.makedirs(output_folder, exist_ok=True) 

features_dict = {}
# Ciclo di estrazione
for img_path in tqdm(image_files, desc="Estrazione feature con EfficientNetV2"):
    image_id = os.path.basename(img_path).split('.')[0]
    # Passa la dimensione corretta alla funzione
    feature_vector = extract_feature(img_path, model, input_size)
    
    if feature_vector is not None:
        features_dict[image_id] = feature_vector

# Salva il nuovo dizionario di feature
output_path = os.path.join(output_folder, "image_features_efficientnet.pkl")
with open(output_path, "wb") as f:
    pickle.dump(features_dict, f)

print(f"\n✔️ Estrazione completata. {len(features_dict)} nuove feature salvate in '{output_path}'.")

Estrazione feature: 100%|██████████| 8091/8091 [46:56<00:00,  2.87it/s]  
